# Pi-Hole on Raspberry Pi

First use `Raspberry Pi Imager` to burn an SD card. Customize the image before burning it.
- set the hostname
- create the user and password
- turn on `ssh` and disable password connections
- **add public ssh keys** for client connections

Once the SD card is ready, plug it into the Raspberry Pi and boot it up.

Login with `ssh`.

In [1]:
%login pihole

[ssh] Login to pihole...
[ssh] host=pihole hostname=10.0.1.11 other_conf={'user': 'samyules', 'port': None, 'keyfile': ['/home/jovyan/.ssh/pi-hole-rsa'], 'load_system_ssh_config': False, 'missing_host_policy': <paramiko.client.WarningPolicy object at 0x7f4e701e9390>}
[ssh] Successfully logged in.


## Install [log2ram](https://github.com/azlux/log2ram)
After first boot, install log2ram. This will increase the lifespan of the SD card by reducing the number of writes. It mounts the log directory to RAM, and writes logs to the SD card once a day.

### Install via APT (recommended)
Reboot after installing

In [ ]:
echo "deb [signed-by=/usr/share/keyrings/azlux-archive-keyring.gpg] http://packages.azlux.fr/debian/ bookworm main" | sudo tee /etc/apt/sources.list.d/azlux.list
sudo wget -O /usr/share/keyrings/azlux-archive-keyring.gpg  https://azlux.fr/repo.gpg
sudo apt update
sudo apt install log2ram

### Check if it is working

In [ ]:
sudo systemctl status log2ram

Or run the following and it should output something similar.

In [2]:
df -h | grep log2ram
# log2ram          40M  532K   40M   2% /var/log

[ssh] host = pihole, cwd = /home/samyules
log2ram          40M   40M  400K 100% /var/log


## Install [Pi-hole](https://pi-hole.net)
Ensure that the you have a static IP address set before running the script.

### Run the script

In [ ]:
curl -sSL https://install.pi-hole.net | bash

After installing you can set pi-hole as your dns for your network.

## Follow On

### Add local wildcard servers to dnsmasq

Pi-hole does not allow setting wildcard DNS records through the web interface. However, it is possible to do manually through a `dnsmasq` config file. Create a config file in `/etc/dnsmasq.d/`. Name it something like `99-reverse-proxy-web-servers.conf`. The contents should look like this:

In [ ]:
address=/revproxy.my.lan/192.168.0.42

This will cause that domain and *all* of it's subdomains to resolve to the same ip address. e.g. `dig plex.revproxy.my.lan` and `dig revproxy.my.lan` will both result in `192.168.0.42`.

Reference: Answer on [discourse.pi-hole.net](https://discourse.pi-hole.net/t/support-wildcards-in-local-dns-records/32098/12)

### Manually set rasperry pi DNS settings

The Raspberry Pi host should first search `localhost` for dns. Here is how to change the settings and set static dns.

Raspberry Pi OS uses `NetworkManager` by default to manage network connections. `nmcli` is included in the distro, and you can use it to manually change dns settings. Here is the command to manually change dns settings for a network connection: (first find `$connectionName` by running `nmcli con`.

In [12]:
nmcli con

[ssh] host = pihole, cwd = /home/samyules
NAME                UUID                                  TYPE      DEVICE 
Wired connection 1  4ded1b11-5abf-3c83-9de0-1316b0e44fa8  ethernet  eth0   
lo                  82e09958-221d-4d79-93de-2d4efd9f7f02  loopback  lo     


In [ ]:
connectionName='Wired connection 1'

In [ ]:
sudo nmcli con mod 'Wired connection 1' ipv4.dns "127.0.0.1 8.8.8.8 8.8.4.4"

If you want to ignore automatically configured nameservers and search domains from the DHCP server then run this:

In [ ]:
sudo nmcli con mod $connectionName ipv4.ignore-auto-dns yes

Not 100% sure what this does, but it's in the [ArchLinux wiki](https://wiki.archlinux.org/title/Dnsmasq).

In [ ]:
sudo nmcli con mod $connectionName ipv4.dns-options trust-ad

Next, rinse and repeat with ipv6.

In [ ]:
sudo nmcli con mod $connectionName ipv6.dns ::1 #ipv6 localhost
sudo nmcli con mod $connectionName ipv6.dns-settings trust-ad
sudo nmcli con mod $connectionName ipv6.ignore-auto-dns yes

To enable the changes, **restart the *NetworkManager* service**.

In [ ]:
sudo systemctl restart NetworkManager

Verify that your settings now appear in `resolv.conf`. Don't make changes manually to this file. It is managed by `NetworkManager`.

In [ ]:
cat /etc/resolv.conf

###### Reference: [ServerFault](https://serverfault.com/a/810639)

### IPV6 Static*(ish)* Addresses
IPV6 is kind of a pain in the @$$. Comcast give me a /64 prefix delegation. Basically the first half of the address belongs to Comcast. The second half of the address is set by my router. Looks like this:

- **2601:681:8b00:7e70**:*3899:8dcd:3aa9:ce90*
- So, you've got the **Comcast Prefix** then the rest *assigned by router*

The problem is that Comcast can (*and probably will*) change the prefix whenever they want. IPV6 is designed to protect privacy. Addresses are not fixed, and the second half of the address will change on an interval, which means that the default configuration will cause the address to drop and get replaced with a new one all the time. I found a semi-solution that I will outline.

##### IPV6 Token
This solves 50% of the problem. I cannot due anything about Comcast changing the prefix. The first half of the IPv6 address will change whenever Comcast wants. However, I can change the privacy mode of the interface, and get a static token for the second half of the address. Like so:

In [ ]:
nmcli con mod $connectionName ipv6.addr-gen-mode eui64
nmcli con mod $connectionName ipv6.token ::deca:fbad:c0:ffee

The token can be whatever you want it to be. It is the last 4 octets of the IPv6 address. 

Now **restart NetworkManager**.

In [ ]:
sudo systemctl restart NetworkManager

Now I have an adress that looks like `2601:681:8b00:7e70:4091:1000:10:111`. The last 4 octets will not change. :botwie:

###### Reference: [ServerFault](https://serverfault.com/a/968644)

In [3]:
%logout

[ssh] Closing existing connection.
[ssh] Successfully logged out.
